In [1]:
from pathlib import Path
from glob import glob
from tqdm import tqdm
import pandas as pd
from pprint import pprint

import nltk

from nltk.tokenize import sent_tokenize

In [2]:
x = [ "fr", "es", "de","pt"]
# x = ["hi"]  # Removed due to sentence tokenization incapability.

files = sorted(list(Path("/home/operation/projects/data/2019_dw/dw-2019/").glob("**/*.jsonl")))
files = [i for i in files if i.parent.stem in x]
files[:10], len(files)

([PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2001_10.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2001_11.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2001_12.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2001_9.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_1.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_10.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_11.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_12.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_2.jsonl'),
  PosixPath('/home/operation/projects/data/2019_dw/dw-2019/de/2002_3.jsonl')],
 814)

In [5]:
pd.read_json(files[0], lines=True)

,feedURL,sourceItemType,sourceItemPageUrl,sourceItemIdAtOrigin,sourceItemOriginFeedName,sourceItemLangeCodeGuess,sourceItemTitle,sourceItemMainText,sourceItemTeaser,sourceItemKeywords,sourceItemPhotoURL,sourceItemDate,sourceItemScenarios
0,http://api.dw.com,Article,https://p.dw.com/p/19WQ,http://api.dw.com/api/detail/article/274934,dw-history-feed,de,Militärischer Aufmarsch rund um Afghanistan,Das Washingtoner Verteidigungsministerium hat ...,Die USA und Großbritannien ziehen nach den Ter...,"[afghanistan, terror, usa, großbritannien, mil...",,2001-10-01 17:15:29,"[DW, SUMMA]"
1,http://api.dw.com,Article,https://p.dw.com/p/18t6,http://api.dw.com/api/detail/article/272496,dw-history-feed,de,Blutbad in Schweizer Kanton Zug,Ein Amokläufer hat im Parlament des Schweizer ...,14 Tote bei Amoklauf in Schweizer Kantonsparla...,"[amok, schweiz, zug, parlament, kanton]",,2001-10-02 11:14:51,"[DW, SUMMA]"
2,http://api.dw.com,Article,https://p.dw.com/p/19ZW,http://api.dw.com/api/detail/article/275126,dw-history-feed,de,"""Tag der offenen Moschee""",Besonders nach den Terroranschlägen in den Ver...,"Die 3,2 Millionen Muslime in Deutschland laden...",[],,2001-10-02 11:13:34,"[DW, SUMMA]"
3,http://api.dw.com,Article,https://p.dw.com/p/19N0,http://api.dw.com/api/detail/article/274350,dw-history-feed,de,NATO sichert den USA Unterstützung zu,Die Mitgliedstaaten der Allianz haben die Wüns...,Die NATO-Verbündeten haben den USA Unterstützu...,[],,2001-10-04 16:52:48,"[DW, SUMMA]"
4,http://api.dw.com,Article,https://p.dw.com/p/19rt,http://api.dw.com/api/detail/article/276265,dw-history-feed,de,Mailänder Moderesümee,Die Defilees führten von Afrika (Jil Sander un...,Die ganze Welt ein Laufsteg: Mailands Designer...,"[mailand, milano, sommermode, damenmode, kolle...",https://api.dw.com/image/275882_4.jpg,2001-10-04 15:53:56,"[DW, SUMMA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,http://api.dw.com,Article,https://p.dw.com/p/1JX0,http://api.dw.com/api/detail/article/313410,dw-history-feed,de,Flügellahmer Ferienflieger,"Das Unternehmen versichert, dass der Flugbetri...","Schwarze Wolken, aber noch kein Gewittersturm ...",[],,2001-10-31 14:59:12,"[DW, SUMMA]"
143,http://api.dw.com,Article,https://p.dw.com/p/1JaQ,http://api.dw.com/api/detail/article/313622,dw-history-feed,de,Das lange Warten auf den Beginn des Regierens,Was die neue bulgarische Regierung zweifellos ...,Eine Bilanz der ersten 100 Tage der neuen bulg...,"[Bulgarien, Simeon, Regierung, Bilanz, Politik]",,2001-10-31 14:41:33,"[DW, SUMMA]"
144,http://api.dw.com,Article,https://p.dw.com/p/1Jla,http://api.dw.com/api/detail/article/314314,dw-history-feed,de,Murren in Brüssel über Prodi,So richtig war es niemandem aufgefallen: Bei d...,Beim EU-Gipfel in Gent kam es hinter den Kulis...,"[Prodi, EU, Belgien, Ratspräsidentschaft, Gent...",,2001-10-31 14:18:56,"[DW, SUMMA]"
145,http://api.dw.com,Article,https://p.dw.com/p/1JiD,http://api.dw.com/api/detail/article/314105,dw-history-feed,de,Baubeginn für Holocaust-Mahnmal in Berlin,Das deutsche Holocaust-Mahnmal wird im Herzen ...,Zehn Jahre lang wurde heftigst darüber debatti...,[],,2001-10-31 11:53:42,"[DW, SUMMA]"


In [27]:
files = sorted(list(Path("/home/operation/projects/notebooks/validation/").glob("**/*.jsonl")))
for file in files:
    df = pd.read_json(file, lines=True)
    text = df['sourceItemMainText']
    tokens, labels = tokenize_document(text)
    with torch.no_grad():
        embeddings = embedder.predict(tokens, source_lang="eng_Latn", batch_size=64)
    save_data = {
        "sentences": tokens,
        "embeddings": embeddings.detach().cpu(),
        "labels": torch.tensor(labels),
    }
    print(torch.tensor(labels).shape, embeddings.shape)
    torch.save(save_data, f"/home/operation/projects/notebooks/validation/{file.stem}.pt")
embeddings.shape

torch.Size([172]) torch.Size([172, 1024])
torch.Size([167]) torch.Size([167, 1024])


torch.Size([167, 1024])

In [22]:
torch.load("/home/operation/projects/notebooks/validation/val_ndtv_sports.pt")

{'embeddings': tensor([[-0.0057,  0.0184, -0.0117,  ..., -0.0051, -0.0055, -0.0093],
         [-0.0034,  0.0215, -0.0080,  ..., -0.0008, -0.0006,  0.0042],
         [ 0.0015,  0.0009, -0.0037,  ..., -0.0074, -0.0071,  0.0056],
         ...,
         [ 0.0053,  0.0101,  0.0044,  ..., -0.0150, -0.0006,  0.0036],
         [-0.0040, -0.0020, -0.0039,  ..., -0.0171, -0.0093, -0.0123],
         [-0.0040, -0.0056, -0.0048,  ...,  0.0042,  0.0025,  0.0107]],
        device='cuda:0'),
 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0,

In [3]:
df = pd.read_json(files[0], lines=True)
df.iloc[0]

feedURL                                                     http://api.dw.com
sourceItemType                                                        Article
sourceItemPageUrl                                     https://p.dw.com/p/19WQ
sourceItemIdAtOrigin              http://api.dw.com/api/detail/article/274934
sourceItemOriginFeedName                                      dw-history-feed
sourceItemLangeCodeGuess                                                   de
sourceItemTitle                   Militärischer Aufmarsch rund um Afghanistan
sourceItemMainText          Das Washingtoner Verteidigungsministerium hat ...
sourceItemTeaser            Die USA und Großbritannien ziehen nach den Ter...
sourceItemKeywords          [afghanistan, terror, usa, großbritannien, mil...
sourceItemPhotoURL                                                           
sourceItemDate                                            2001-10-01 17:15:29
sourceItemScenarios                                             

In [17]:
def tokenizer(text: str, lang: str="en"):
    """
    Improve this logic later for multiple languages
    """
    nltk_tokenizer_lang = ["en", "fr", "es", "de", "pt"]
    if lang in nltk_tokenizer_lang:
        tokens = sent_tokenize(text)
    return tokens

def tokenize_document(list_docs: list, lang="en"):
    tokens = []
    labels = []
    for t in list_docs:
        toks = tokenizer(t, lang=lang)
        tokens+=toks
        labels+=[0]*len(toks)
        labels[-1]=1
    return tokens, labels


# tokens = tokenizer(df.iloc[0]["sourceItemMainText"], lang="en")
# labels = [0] * len(tokens)
# labels[-1] = 1
# labels

toks, labs = tokenize_document(df['sourceItemMainText'].tolist(), lang="en")
len(toks), len(labs)

(167, 167)

In [3]:
import torch
from torch import nn
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline

In [4]:
from sonar.models.sonar_speech.loader import load_sonar_speech_model
from sonar.models.sonar_text import (
    load_sonar_text_decoder_model,
    load_sonar_text_encoder_model,
    load_sonar_tokenizer,
)

In [5]:
device = torch.device("cuda")

In [6]:
t2enc = load_sonar_text_encoder_model("text_sonar_basic_encoder", device=device).eval()
text_tokenizer = load_sonar_tokenizer("text_sonar_basic_encoder")

Using the cached checkpoint of text_sonar_basic_encoder. Set `force` to `True` to download again.


control_symbols=['__ace_Arab__', '__ace_Latn__', '__acm_Arab__', '__acq_Arab__', '__aeb_Arab__', '__afr_Latn__', '__ajp_Arab__', '__aka_Latn__', '__amh_Ethi__', '__apc_Arab__', '__arb_Arab__', '__ars_Arab__', '__ary_Arab__', '__arz_Arab__', '__asm_Beng__', '__ast_Latn__', '__awa_Deva__', '__ayr_Latn__', '__azb_Arab__', '__azj_Latn__', '__bak_Cyrl__', '__bam_Latn__', '__ban_Latn__', '__bel_Cyrl__', '__bem_Latn__', '__ben_Beng__', '__bho_Deva__', '__bjn_Arab__', '__bjn_Latn__', '__bod_Tibt__', '__bos_Latn__', '__bug_Latn__', '__bul_Cyrl__', '__cat_Latn__', '__ceb_Latn__', '__ces_Latn__', '__cjk_Latn__', '__ckb_Arab__', '__crh_Latn__', '__cym_Latn__', '__dan_Latn__', '__deu_Latn__', '__dik_Latn__', '__dyu_Latn__', '__dzo_Tibt__', '__ell_Grek__', '__eng_Latn__', '__epo_Latn__', '__est_Latn__', '__eus_Latn__', '__ewe_Latn__', '__fao_Latn__', '__pes_Arab__', '__fij_Latn__', '__fin_Latn__', '__fon_Latn__', '__fra_Latn__', '__fur_Latn__', '__fuv_Latn__', '__gla_Latn__', '__gle_Latn__', '__glg_

Using the cached tokenizer of text_sonar_basic_encoder. Set `force` to `True` to download again.


In [7]:
embedder = TextToEmbeddingModelPipeline(t2enc, text_tokenizer, device=device)

In [8]:
with torch.no_grad():
    embeddings = embedder.predict(toks, source_lang="eng_Latn", batch_size=64)

NameError: name 'toks' is not defined

In [22]:
embeddings.shape

torch.Size([3878, 1024])

In [89]:
class MyModel(nn.Module):
    def __init__(self, encoder="text_sonar_basic_encoder", tokenizer="text_sonar_basic_encoder", device="cuda") -> None:
        super().__init__()
        if isinstance(device, str): 
            device = torch.device(device)
        t2enc = load_sonar_text_encoder_model(encoder, device=device).eval()
        text_tokenizer = load_sonar_tokenizer(tokenizer)
        self.backbone = TextToEmbeddingModelPipeline(t2enc, text_tokenizer, device=device)
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.head = nn.Sequential(
                        nn.Linear(1024, 512),
                        nn.ReLU(),
                        nn.Linear(512, 512),
                        nn.ReLU(),
                        nn.Linear(512, 2)
                    )
        self.head.to(device)

    def forward(self, sentence_list, source_lang="eng_Latn"):
        embeddings = self.backbone.predict(sentence_list, source_lang=source_lang, batch_size=64)
        predictions = self.head(embeddings)
        return predictions

class MyModel2(nn.Module):
    def __init__(self, device="cuda") -> None:
        super().__init__()
        if isinstance(device, str): 
            device = torch.device(device)
        self.head = nn.Sequential(
                        nn.Linear(1024, 512),
                        nn.ReLU(),
                        nn.Linear(512, 512),
                        nn.ReLU(),
                        nn.Linear(512, 2)
                    )
        self.head.to(device)

    def forward(self, embeddings):
        predictions = self.head(embeddings)
        return predictions
model = MyModel2()

In [86]:
with torch.no_grad():
    embeddings = embedder.predict(toks, source_lang="eng_Latn", batch_size=64)
    out = model(embeddings)
out.shape

torch.Size([3878, 2])

In [32]:
def tokenizer(text: str, lang: str="en"):
    """
    Improve this logic later for multiple languages
    """
    nltk_tokenizer_lang = ["en", "fr", "es", "de", "pt"]
    if lang in nltk_tokenizer_lang:
        tokens = sent_tokenize(text)
    return tokens

def tokenize_document(list_docs: list, lang="en"):
    tokens = []
    labels = []
    for t in list_docs:
        toks = tokenizer(t, lang=lang)
        tokens+=toks
        labels+=[0]*len(toks)
        labels[-1]=1
    return tokens, labels


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, languages: list=["en", "fr", "es", "de", "pt"]):
        self.data_dir = data_dir
        self.languages = languages
        files = sorted(list(Path(self.data_dir).glob("**/*.jsonl")))
        self.files = [i for i in files if i.parent.stem in self.languages]
        print(f"Found {len(self.files)} files in {self.data_dir} having following langagues: {set([i.parent.stem for i in self.files])}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        df = pd.read_json(self.files[idx], lines=True)
        x, y = tokenize_document(df['sourceItemMainText'].tolist(), lang="en") 
        return x, y

In [33]:
dataset = CustomDataset(data_dir="/home/operation/projects/data/2019_dw/dw-2019")

Found 1027 files in /home/operation/projects/data/2019_dw/dw-2019 having following langagues: {'en', 'es', 'pt', 'fr', 'de'}


In [76]:
x ,y = dataset[1]
with torch.no_grad():
    out = model(x)
out.shape

torch.Size([6858, 2])

In [52]:
sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6

0.788482

In [96]:
optimizer = optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
data = next(iter(dataset))
inp, labl = data
with torch.no_grad():
    embeddings = embedder.predict(inp, source_lang="eng_Latn", batch_size=64)

embb = torch.tensor(embeddings)
outputs = model(embb)
outputs.shape, labl

In [99]:
loss_fn(outputs, torch.tensor(labl).to(device))

tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:

for i, data in enumerate(dataset):
    inp, labels = data
    optimizer.zero_grad()
    with torch.no_grad():
        embeddings = embedder.predict(inp, source_lang="eng_Latn", batch_size=64)
        embb = torch.tensor(embeddings)
    outputs = model(embb)
    loss = loss_fn(outputs, torch.tensor(labl).to(device))
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 10 == 0:
        last_loss = running_loss / 1000 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        tb_x = epoch_index * len(training_loader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.